In [9]:
import sys

sys.path.append('modules/')
from modules import goodreads, notion

# Provide the Goodreads URL to capture the metadata for
URL = "https://www.goodreads.com/book/show/55275019-machine-learning-design-patterns?ref=nav_sb_ss_1_111"

book = await goodreads.async_get_book_metadata(URL, verbose=True)

Name: Machine Learning Design Patterns: Solutions to Common Challenges in Data Preparation, Model Building, and MLOps
Image: https://images-na.ssl-images-amazon.com/images/S/compressed.photo.goodreads.com/books/1599583594i/55275019.jpg
Format: Paperback
Number of pages: 400
Language: English
ISBN: 9781098115784
Rating: 4.2
Number of ratings: 147
Number of reviews: 21
Authors: ['Valliappa Lakshmanan', 'Sara Robinson', 'Michael Munn']


In [12]:
notion.post_book_from_goodreads(book)

<Response [400]>

In [ ]:
import scrapy
from scrapy.crawler import CrawlerProcess, CrawlerRunner
from multiprocessing import Process, Queue
from twisted.internet import reactor

class GoodreadsSpider(scrapy.Spider):
    name = 'goodreads'
    start_urls = ['https://www.goodreads.com/book/show/55275019-machine-learning-design-patterns?ref=nav_sb_ss_1_111']

    def parse(self, response):
        book_title = response.css('#bookTitle::text').get()
        book_author = response.css('a.authorName>span::text').get()
        book_cover = response.css('.bookCover::attr(src)').get()
        book_rating = response.css('span[itemprop=ratingValue]::text').get()
        book_isbn = response.css('div.infoBoxRowItem[itemprop=isbn]::text').get()
        book_language = response.css('div[itemprop=inLanguage]::text').get()

        print(book_title)
        print(book_author)
        print(book_cover)
        yield {
            'title': book_title,
            'author': book_author,
            'cover': book_cover,
            'rating': book_rating,
            'isbn': book_isbn,
            'language': book_language
        }


# def f(q, spider):
#     try:
#         runner = CrawlerRunner()
#         deferred = runner.crawl(spider)
#         deferred.addBoth(lambda _: reactor.stop())
#         reactor.run()
#         q.put(None)
#     except Exception as e:
#         q.put(e)
#
#     return q

# the wrapper to make it run more times
# def run_spider(spider):
#
#     queue = Queue()
#     process = Process(target=f, args=(queue, spider))
#     process.start()
#     result = queue.get()
#     process.join()
#
#     if result is not None:
#         raise result

# Run the spider
# run_spider(GoodreadsSpider)
process = CrawlerProcess()
process.crawl(GoodreadsSpider)
process.start()


In [ ]:
from requests_html import AsyncHTMLSession
from bs4 import BeautifulSoup


URL = "https://www.goodreads.com/book/show/55275019-machine-learning-design-patterns?ref=nav_sb_ss_1_111"

asession = AsyncHTMLSession()
r = await asession.get(URL)
await r.html.arender()

soup = BeautifulSoup(r.html.raw_html, "lxml")

In [ ]:
from modules.utils import GoodReadBook
import json
script = soup.find('script', {'type': 'application/ld+json'}).text
book = GoodReadBook(json.loads(script), URL)


In [ ]:
book.name